In [1]:
!git init internal

Initialized empty Git repository in /Users/royan/git/knockdata.github.io/notebooks/internal/.git/


In [3]:
!cd internal

/bin/sh: line 0: cd: internal: No such file or directory


In [6]:
!find .git

.git
.git/config
.git/objects
.git/objects/pack
.git/objects/info
.git/HEAD
.git/info
.git/info/exclude
.git/description
.git/hooks
.git/hooks/commit-msg.sample
.git/hooks/pre-rebase.sample
.git/hooks/pre-commit.sample
.git/hooks/applypatch-msg.sample
.git/hooks/fsmonitor-watchman.sample
.git/hooks/pre-receive.sample
.git/hooks/prepare-commit-msg.sample
.git/hooks/post-update.sample
.git/hooks/pre-merge-commit.sample
.git/hooks/pre-applypatch.sample
.git/hooks/pre-push.sample
.git/hooks/update.sample
.git/refs
.git/refs/heads
.git/refs/tags


In [7]:
!rm -rf .git/hooks

## Monitor file changes

Run the following command in a separate command 

    while true; do clear; tree .git; sleep 1; done;
    
Might need install tree command

    brew install tree

## Nothing yet


    >> git log
    fatal: your current branch 'master' does not have any commits yet
    
    >> git status
    On branch master

    No commits yet

    nothing to commit (create/copy files and use "git add" to track)

## Create Object

Create an object

    >> echo "first line" | git hash-object -w --stdin
    08fe2720d8e3fe3a5f81fbb289bc4c7a522f13da
    
An object shall be created under `.git/objects/08/fe...`

Get content by command, we only need first few hash id if it's identical

    git cat-file -p 08fe
    
Get the type by command, which will get back type `blob`

    git cat-file -t 08fe

## Add Index
    
We only have an object now. Add to index by command. we need the full sha1 this time. 

    git update-index --add --cacheinfo 100644 08fe2720d8e3fe3a5f81fbb289bc4c7a522f13da file1.txt
    
If we run git status, it will say. 

    Changes to be committed:
      (use "git rm --cached <file>..." to unstage)
        new file:   file1.txt

    Changes not staged for commit:
      (use "git add/rm <file>..." to update what will be committed)
      (use "git restore <file>..." to discard changes in working directory)
        deleted:    file1.txt
        
Now the file has added to the index. We can see file1.txt is in file .git/index. Though there are lots of strange characters.

    cat .git/index

## Add to tree

Add the index with the tree by command

    >> git write-tree
    
    a156bcae5181f14958609bc0a6337fe78e8f8ba9
    
A new file added in `.git/objects/a1/56..`   

Add the index with the tree by command

    >> git cat-file -t a156
    tree
    
    >> git cat-file -p a156
    100644 blob 08fe2720d8e3fe3a5f81fbb289bc4c7a522f13da	file1.txt

The content is in binary format. By cat will not see reasonable text.

    >> cat objects/a1/56bcae5181f14958609bc0a6337fe78e8f8ba9 

## Add commit


    >> git commit-tree a156bcae5181f14958609bc0a6337fe78e8f8ba9 -m "first file"
    12418a43a7ec0b243f1e391e175e971aef4f3a14
    
A new file added in `.git/objects/12/41...`



## HEAD

HEAD is now point to refs/heads/master. 

    >> cat .git/HEAD 
    ref: refs/heads/master
    
While there is no master yet. update master to point to previous commit with command.

    git update-ref refs/heads/master 12418a43a7ec0b243f1e391e175e971aef4f3a14
    
This is actually is a pointer.

    >> cat .git/refs/heads/master 
    12418a43a7ec0b243f1e391e175e971aef4f3a14
    
    
Now we can run git log to get the first commit.

    git log

## Update working directory

It will report `deleted:    file1.txt` if we run git status. The reason is that we only add the to repo but have not added to the working directory. 

We can pull the content in the repo to the working directory by. -- is a separator between command and files.

    >> git checkout HEAD -- file1.txt
    
or just by 

    >> git reset --hard

## The normal way

    git init
    rm -rf .git/hooks
    echo "first line" >> file1.txt
    chmod 100644 file1.txt
    git add .
    git commit -m "first file"
    
    
The tree's SHA1 will be different due to timestamp different. other objects shall have the same SHA1